In [ ]:
# Missing Values

# Three Approaches

# 1. A Simple Option: Drop Columns with Missing Values
# 2. A Better Option: Imputation
# 3. An Extension To Imputation

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

melbourne_file_path = '../Intro to Machine Learning/dataset/melb_data.csv'
# Load the data
data = pd.read_csv(melbourne_file_path)

# Select target
y = data.Price

# To keep things simple, we'll use only numerical predictors
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2)

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
9457,1,5.2,3055.0,1.0,1.0,1.0,406.0,40.0,1970.0,-37.75847,144.94226,7082.0
9615,5,14.6,3093.0,5.0,2.0,2.0,864.0,NaN,NaN,-37.72754,145.11021,1624.0
477,3,10.5,3034.0,3.0,1.0,1.0,597.0,NaN,NaN,-37.75680,144.85720,4502.0
5459,6,11.4,3084.0,3.0,3.0,4.0,911.0,NaN,NaN,-37.73120,145.06350,3540.0
1832,3,9.3,3162.0,3.0,2.0,2.0,296.0,NaN,1910.0,-37.89060,145.02240,5051.0


In [8]:
# Define Function to Measure Quality of Each Approach

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [11]:
# Approach 1: Drop Columns with Missing Values
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):
174889.21726926853


In [12]:
# Approach 2: Imputation
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
172984.5145557192
